In [ ]:
pip install python-dotenv PyPDF2 streamlit langchain transformers datasets faiss-cpu faiss-gpu

In [ ]:
from dotenv import load_dotenv
import os
from PyPDF2 import PdfReader,PdfFileReader
import streamlit as st
from langchain.text_splitter import CharacterTextSplitter
from langchain import FAISS

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
import numpy as np

# Replace 'yiyanghkust/finbert-tone', with the actual model name you want to use
model_name = 'yiyanghkust/finbert-tone'

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Define a function to create FinBERT embeddings for a given text
def create_finbert_embeddings(text):
    # Tokenize the text
    tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

    # Forward pass through the model
    with torch.no_grad():
        outputs = model(**tokens)

    # Extract the embeddings from the model's output
    embeddings = outputs.last_hidden_state

    # Average the token embeddings to get a single vector representation
    average_embedding = torch.mean(embeddings, dim=1).numpy()

    return average_embedding

# Example usage
text = "Financial results for the last quarter are promising."
embedding = create_finbert_embeddings(text)

# Now, 'embedding' contains the FinBERT embeddings for the given text
print(embedding.shape)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


(1, 768)


In [ ]:
def process_text(text):
    # Split the text into chunks using Langchain's CharacterTextSplitter
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)

    # Convert the chunks of text into embeddings using FinBERTEmbeddings
    embeddings = create_finbert_embeddings()
    knowledgeBase = FAISS.from_texts(chunks, embeddings)

    return knowledgeBase

In [ ]:
# from textsplitter import CharacterTextSplitter
# from knowledge_base import FAISS

# Replace 'yiyanghkust/finbert-tone' with the actual model name
model_name = 'yiyanghkust/finbert-tone'

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Define the function to create FinBERT embeddings
def create_finbert_embeddings(text):
    tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**tokens)
    embeddings = outputs.last_hidden_state
    average_embedding = torch.mean(embeddings, dim=1).numpy()
    return average_embedding

def process_text(text):
    # Split the text into chunks using Langchain's CharacterTextSplitter
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)

    # Create a list of embeddings for each chunk
    chunk_embeddings = [create_finbert_embeddings(chunk) for chunk in chunks]

    # Convert the list of embeddings into a knowledge base using FAISS
    knowledgeBase = FAISS.from_texts(chunk_embeddings)

    return knowledgeBase


In [ ]:
# dir(FAISS)

In [ ]:
!ls

2016_Annual_Report.pdf	sample_data


In [ ]:
pdf_file = "2016_Annual_Report.pdf"
pdf_reader = PdfReader(pdf_file)
text = ""
for page in pdf_reader.pages[15:26]:
  text += page.extract_text()

In [ ]:
pdf_reader.pages[23].mediabox

RectangleObject([18, 18, 612, 792])

In [ ]:
text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
chunks = text_splitter.split_text(text)

In [ ]:
chunk_embeddings = [create_finbert_embeddings(chunk) for chunk in chunks]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# !pip install

In [ ]:
from datasets import Dataset

In [ ]:
from transformers import AutoTokenizer, TFAutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFMPNetModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFMPNetModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMPNetModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFMPNetModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMPNetModel for predictions without further training.


In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="tf"
    )
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [ ]:
type(chunks),len(chunks)

(list, 46)

In [ ]:
custom_dataset["text"][0]

'Entergy Corporation and Subsidiaries\nManagement’s Financial Discussion and Analysis\nResults of Operations\n2016 Compared to 2015\nFollowing are income statement variances for Utility, Entergy Wholesale Commodities, Parent & Other, and \nEntergy comparing 2016 to 2015 showing how much the line item increased or (decreased) in comparison to the prior \nperiod.\nUtilityEntergy\nWholesale\nCommoditiesParent &\nOther (a) Entergy\n(In Thousands)\n2015 Consolidated Net Income (Loss) $1,114,516 ($1,065,657) ($205,593) ($156,734)\nNet revenue (operating revenue less fuel expense,\npurchased power, and other regulatory charges/\ncredits) 350,528 (123,791) (33) 226,704\nOther operation and maintenance (83,265) 15,269 9,726 (58,270)\nAsset write-offs, impairments, and related charges (68,672) 799,403 — 730,731\nTaxes other than income taxes (10,229) (16,259) (432) (26,920)\nDepreciation and amortization 49,600 (39,180) (509) 9,911\nGain on sale of asset — (154,037) — (154,037)\nOther income 15,

In [ ]:
create_finbert_embeddings(custom_dataset["text"][0]).shape

(1, 768)

In [ ]:
data = {
    "text": chunks
}

# Create a Hugging Face dataset
custom_dataset = Dataset.from_dict(data)

# Map the "embeddings" column
embeddings_dataset = custom_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).numpy()[0]}
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


Map:   0%|          | 0/46 [00:00<?, ? examples/s]

In [ ]:
# embeddings_dataset[0]["embeddings"]

In [ ]:
embeddings_dataset.add_faiss_index(column="embeddings", index_name="faiss_index")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['text', 'embeddings'],
    num_rows: 46
})

In [ ]:
question = "What is the net change in net revenue during 2015 for entergy corporation?"
question_embedding = get_embeddings([question]).numpy()
question_embedding.shape

(1, 768)

In [ ]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "faiss_index", question_embedding, k=5
)

In [ ]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [ ]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [ ]:
samples_df["text"][3]

'Amount\n(In Millions)\n2014 net revenue $2,224\nNuclear realized price changes (310)\nVermont Yankee shutdown in December 2014 (305)\nNuclear volume, excluding Vermont Yankee effect 20\nOther 37\n2015 net revenue $1,666\nAs shown in the table above, net revenue for Entergy Wholesale Commodities decreased by approximately \n$558 million in 2016 primarily due to:\n• lower realized wholesale energy prices, primarily due to significantly higher Northeast market power prices\nin 2014, and lower capacity prices in 2015; and\n• a decrease in net revenue as a result of Vermont Yankee ceasing power production in December 2014.\nThe decrease was partially offset by higher volume in the Entergy Wholesale Commodities nuclear fleet, excluding \nVermont Yankee, resulting from fewer refueling outage days in 2015 as compared to 2014, partially offset by more \nunplanned outage days in 2015 as compared to 2014.\n22Entergy Corporation and Subsidiaries\nManagement’s Financial Discussion and Analysis'

In [ ]:
# !apt install libomp-dev
# !pip install faiss-cpu

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libomp-dev is already the newest version (1:14.0-55~exp2).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 26.4 MB/s eta 0:00:00


ImportError: ignored

In [ ]:
HuggingFaceInferenceAPIEmbeddings

In [ ]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=inference_api_key,
    model_name=model_name
)

# query_result = embeddings.embed_query(text)
# query_result[:3]

NameError: ignored

In [ ]:
FAISS.from_embeddings()

TypeError: ignored

In [ ]:
embeddings =
db = FAISS.from_texts(chunks, embeddings)

NameError: ignored

In [ ]:
knowledgeBase = process_text(text)

TypeError: ignored

In [ ]:
query = "Question : what is the net change in net revenue during 2015 for entergy corporation?"

In [ ]:
knowledgeBase.similarity_search(query)